<h3>llm_json_structure_test</h3>

<h4>1. raw를 그대로 LLM에 대입 </h4>

In [20]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("GROQ_API_KEY")

JSON_SCHEMA_EXAMPLE = """
{
  "year": 2021,
  "university": "건국대학교",
  "colleges": [
    {
      "college_name": "",
      "departments": [
        {
          "department_name": "",
          "credits": {
            "general_education": 0,
            "major_required": 0,
            "major_elective": 0,
            "major_total": 0,
            "double_major": 0,
            "graduation_total": 0
          }
        }
      ]
    }
  ]
}
"""

def text_to_json_with_llm(raw_text):
    prompt = f"""
당신은 데이터 구조화 전문가입니다.
아래 텍스트를 JSON으로 변환하세요.

형식 예시는 다음과 같습니다:
{JSON_SCHEMA_EXAMPLE}

텍스트:
{raw_text}
"""
    res = requests.post(
        "https://api.groq.com/openai/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {API_KEY}"
        },
        json={
            "model": "llama-3.1-8b-instant",
            "messages": [
                {"role": "user", "content": prompt}
            ]
        }
    )

    return res.json()["choices"][0]["message"]["content"]


In [27]:
import sys
sys.path.append(os.path.abspath(".."))
from src.utils.pdf_utils import pdf_to_text

raw_text = pdf_to_text(2021, 5)
result = text_to_json_with_llm(raw_text)
print(result)

아래는 위 텍스트를 JSON으로 변환한 예시입니다.

```json
{
  "year": 2021,
  "university": "건국대학교",
  "colleges": [
    {
      "college_name": "산업디자인학과",
      "departments": [
        {
          "department_name": "산업디자인학과",
          "credits": {
            "general_education": 35,
            "major_required": 14,
            "major_elective": 58,
            "major_total": 72,
            "double_major": 40,
            "graduation_total": 132
          }
        }
      ]
    },
    {
      "college_name": "실내디자인학과",
      "departments": [
        {
          "department_name": "실내디자인학과",
          "credits": {
            "general_education": 35,
            "major_required": 13,
            "major_elective": 56,
            "major_total": 69,
            "double_major": 40,
            "graduation_total": 132
          }
        }
      ]
    },
    {
      "college_name": "패션디자인학과",
      "departments": [
        {
          "department_name": "패션디자인학과",
          "credits": {
            "gen

<h4>2. llm-result_issue, college_name_issue 해결을 위한 프롬프트 규칙 추가</h4>

In [25]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("GROQ_API_KEY")

JSON_SCHEMA_EXAMPLE = """
{
  "year": 2021,
  "university": "건국대학교",
  "colleges": [
    {
      "college_name": "",
      "departments": [
        {
          "department_name": "",
          "credits": {
            "general_education": 0,
            "major_required": 0,
            "major_elective": 0,
            "major_total": 0,
            "double_major": 0,
            "graduation_total": 0
          }
        }
      ]
    }
  ]
}
"""

def text_to_json_with_llm_ver2(raw_text):
    prompt = f"""
당신은 데이터 구조화 전문가입니다.
아래 텍스트를 JSON으로 변환하세요.

형식 예시는 다음과 같습니다:
{JSON_SCHEMA_EXAMPLE}

텍스트:
{raw_text}

단과대학 이름은 공백 없이 합쳐진 형태여야 합니다. 
예: "디자인대학","인문사회융합대학", "과학기술대학", "디자인대학" 
"인문", "사회", "융합", "대학"처럼 개별 단어로는 절대로 분리하지 마십시오. 
"응용 화학 과", "스포츠 건강학 과"처럼 학과명과 단과대학명에 공백을 포함하지 마십시오.
부연 설명 없이 JSON만 출력하십시오.
"""
    res = requests.post(
        "https://api.groq.com/openai/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {API_KEY}"
        },
        json={
            "model": "llama-3.1-8b-instant",
            "messages": [
                {"role": "user", "content": prompt}
            ]
        }
    )

    return res.json()["choices"][0]["message"]["content"]


<h4>3. 프롬프트 제약 제거 후 parsing된 text 대입</h4>

In [1]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv("GROQ_API_KEY")

JSON_SCHEMA_EXAMPLE = """
{
  "year": 2021,
  "university": "건국대학교",
  "colleges": [
    {
      "college_name": "",
      "departments": [
        {
          "department_name": "",
          "credits": {
            "general_education": 0,
            "major_required": 0,
            "major_elective": 0,
            "major_total": 0,
            "double_major": 0,
            "graduation_total": 0
          }
        }
      ]
    }
  ]
}
"""

def text_to_json_with_llm_ver2(raw_text):
    prompt = f"""
당신은 데이터 구조화 전문가입니다.
아래 텍스트를 JSON으로 변환하세요.

형식 예시는 다음과 같습니다:
{JSON_SCHEMA_EXAMPLE}

텍스트:
{raw_text}

부연 설명 없이 JSON만 출력하십시오.
"""
    res = requests.post(
        "https://api.groq.com/openai/v1/chat/completions",
        headers={
            "Authorization": f"Bearer {API_KEY}"
        },
        json={
            "model": "llama-3.1-8b-instant",
            "messages": [
                {"role": "user", "content": prompt}
            ]
        }
    )

    return res.json()["choices"][0]["message"]["content"]


In [ ]:
import sys
sys.path.append(os.path.abspath(".."))
from src.parsing.newline_normalizer import newline_normalize
from src.sources.pdf_reader import pdf_to_text

raw_text = pdf_to_text(2021, 5)
parsing_text = newline_normalize(raw_text)
result = text_to_json_with_llm_ver2(parsing_text)
print(result)

ModuleNotFoundError: No module named 'src.utils.pdf_utils'